In [ ]:
from src.features.simple_feature_calculator import SimpleFeatureCalculator
from research.model_pick.candle_fetch import FusionCandles
from strategies.BinanceBtcDeapV1Voting.models.config import FEAT_FRACDIFF, ALL_RAW_FEAT, SSMContainer
from jesse.helpers import date_to_timestamp
import lightgbm as lgb
import pandas as pd
import numpy as np

candle_container = FusionCandles(
    exchange="Binance Perpetual Futures", symbol="BTC-USDT", timeframe="1m"
)
candles = candle_container.get_candles("2024-07-01", "2025-10-01")

feat_calc = SimpleFeatureCalculator()
feat_calc.load(candles, sequential=True)
print("getting raw features...")
df_raw_feats = pd.DataFrame.from_dict(feat_calc.get(ALL_RAW_FEAT))
df_raw_feats.set_index(candles[:, 0].astype(int), inplace=True)
print(f"{df_raw_feats.shape = }")

model_deep_ssm = SSMContainer(model_type="deep_ssm")
model_lg_ssm = SSMContainer(model_type="lg_ssm")

df_deep_ssm = model_deep_ssm.transform(df_raw_feats[FEAT_FRACDIFF])
df_lg_ssm = model_lg_ssm.transform(df_raw_feats[FEAT_FRACDIFF])
df_feat_full = pd.concat([df_deep_ssm, df_lg_ssm, df_raw_feats], axis=1)

test_features = df_feat_full[
    (df_feat_full.index.to_numpy() >= date_to_timestamp("2025-03-01")) &
    (df_feat_full.index.to_numpy() < date_to_timestamp("2025-09-30"))
]

In [ ]:
import json

MODEL_NAME = "c_L5_N1"

with open("strategies/BinanceBtcDeapV1Voting/models/feature_info.json", "r") as f:
    feat_info = json.load(f)
df_selected_feats = test_features[feat_info[f"{MODEL_NAME}"]]

timestamps = candles[:, 0].astype(np.int64)
mask = (timestamps >= test_features.index[0]) & (timestamps <= test_features.index[-1])

datetimes = pd.to_datetime(test_features.index, unit="ms")

model = lgb.Booster(model_file=f"strategies/BinanceBtcDeapV1Voting/models/model_{MODEL_NAME}.txt")
pred = model.predict(df_selected_feats)
assert len(pred) == len(datetimes)

In [ ]:
vol_diff = np.zeros(candles.shape[0])
vol_diff[1:] = candles[1:, 2] - candles[:-1, 2]
vol_diff = vol_diff[mask]
vol_diff = vol_diff * ((pred > 0.5) * 2 - 1)

In [ ]:
from research.plot import analyze_confidence_slices

slice_res = analyze_confidence_slices(
    datetimes,
    pred,
    vol_diff,
    lower_bound=0,
    upper_bound=1,
    threshold=0.5,
)

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# data_1 = pd.read_csv('model_l4_scores.csv')          ############   测试文件
data_1 = df_tmp

data_1_size = data_1.shape[0]     ###### 测试数据行数  ###############
m_size = 25     ####### 测试多少个月 #######
buy = 1     ##### 多 ###################
sell = 0     ##### 空 ####################
rrr = 0.25     ###### 系数 ###################
m = 1000     ###### 总资金 ###################

In [ ]:
ft = 0.01   ### 0.1, 0.05, 0.01

if ft == 0.1:
    da = 1
    xiao = 0.9
    z = 5

if ft == 0.05:
    da = 1
    xiao = 0.95
    z = 10
    
if ft == 0.01:
    da = 1
    xiao = 0.99
    z = 50


for i in range(z):        

    data_1['final'] = 0

    mask = (data_1['score'] < da) & (data_1['score'] >= xiao)
    data_1['final'] = np.where(mask, data_1['volume'], 0)
    
    true_count = mask.sum() / data_1_size 
    
    ### 更新置信度
    da = round((da - ft), 2)
    xiao = round((xiao - ft), 2)

    data_1['high'] = data_1['final'].cumsum()
    data_1['open'] = rrr * data_1['high'] / m

    fig=plt.figure(figsize=(20,10))  
    plt.plot(data_1['eob'], data_1['open'])
    plt.xticks(range(0,data_1_size,int(data_1_size/m_size)))   
    fig.autofmt_xdate()
    plt.grid(1)
    plt.savefig("./temp/" + str(da) + "_" + str(true_count.round(2)) + ".jpg")   
    plt.close()